In [19]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import time
import json
import re

In [12]:
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Acessa a chave de API do Gemini
GEMINI_API_KEY  = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

# Define o modelo
generation_config = {
  "temperature": 0.3,
}

safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

instruction = """You are an expert medical professional, equipped with comprehensive knowledge of human diseases and their symptoms
You create correct and simple symptoms list that is easy to understand.
Without title
Without comments
"""

model = genai.GenerativeModel(model_name="gemini-1.5-pro",
                              generation_config=generation_config,
                              safety_settings=safety_settings,
                              system_instruction=instruction)

In [5]:
def create_few_shot_from_json(file_path, limit=10):
    """
    Carrega exemplos few-shot a partir de um arquivo JSON, com limite opcional.

    Args:
        file_path: O caminho para o arquivo JSON contendo os dados.
        limit: O número máximo de exemplos a serem retornados (padrão: 10).

    Returns:
        Uma lista de strings formatadas para few-shot learning, limitada ao número especificado.
    """
    try:
        with open(file_path, 'r') as file:
            data_dict = json.load(file)

        few_shot_examples = [
            f"{tipo}: {texto}"
            for key, values in data_dict.items()
            for tipo, texto in [("input", key), ("output", ", ".join(map(str, values)))]
        ][:limit]  # Limita a lista aos primeiros 'limit' itens
        return few_shot_examples
    except FileNotFoundError:
        print(f"Arquivo não encontrado: {file_path}")
        return []
    except json.JSONDecodeError:
        print(f"Erro ao decodificar o JSON em: {file_path}")
        return []

In [6]:
# Uso da função
few_shot = create_few_shot_from_json('filtered_diseases_symptoms_Medlineplus.json')
few_shot[:2]


['input: HIV',
 'output: Fever, Chills, Rash, Night sweats (heavy sweating during sleep), Muscle aches, Sore throat, Fatigue, Swollen lymph nodes, Mouth ulcers']

In [7]:
def load_diseases_from_json(file_path):
    """
    Carrega as doenças de um arquivo JSON, que pode ser uma lista ou um dicionário.

    Args:
        file_path (str): O caminho para o arquivo JSON.

    Returns:
        list: Uma lista contendo as doenças.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        if isinstance(data, list):  # Verifica se é uma lista
            return data  # Se for lista, retorna diretamente
        elif isinstance(data, dict):  # Verifica se é um dicionário
            return list(data.keys())  # Se for dicionário, retorna as chaves

    except FileNotFoundError:
        print(f"Erro: Arquivo '{file_path}' não encontrado.")
        return []
    except json.JSONDecodeError:
        print(f"Erro: Falha ao decodificar o JSON em '{file_path}'.")
        return []


In [8]:
# Uso da função
diseases = load_diseases_from_json('empty_symptoms_keys_Medlineplus.json')
print(diseases[:5])  # Exibe as primeiras 10 doenças

['Aspergillosis', 'Abscess', 'Acute Bronchitis', 'Viral Infections', 'Vaccines']


In [36]:
def generate_symptoms_with_ai(model, few_shot, diseases, max_diseases=None, delay=20):
    """
    Gera sintomas para doenças usando um modelo de IA.

    Args:
        model: O modelo de IA a ser usado.
        few_shot: Exemplos few-shot para o modelo.
        diseases: Uma lista de doenças para as quais gerar sintomas.
        max_diseases: Número máximo de doenças a processar (opcional).
        delay: Tempo de espera entre as chamadas à API (em segundos).

    Returns:
        dict: Um dicionário onde as chaves são as doenças e os valores são listas de sintomas.
    """

    diseases_symptoms_ai = {}
    diseases_to_process = diseases if max_diseases is None else diseases[:max_diseases]

    for disease in diseases_to_process:
        output_text = few_shot + [f'input: {disease}', 'output:']
        
        try:
            response = model.generate_content(output_text)
            if not response.parts or len(response.parts) != 1 or "text" not in response.parts[0]:
                raise ValueError("A resposta não contém uma parte de texto válida.")
            
            symptoms = re.sub(r"[^a-zA-Z\s,]", "", response.text)
            symptoms_list = [symptom.strip() for symptom in symptoms.split(', ')]
            diseases_symptoms_ai[disease] = symptoms_list
        except (AttributeError, TypeError, ValueError) as e:
            print(f"Erro ao gerar sintomas para '{disease}': {e}")
        
        time.sleep(delay)

    return diseases_symptoms_ai

In [37]:
# Uso da função (exemplo)
diseases_symptoms_ai = generate_symptoms_with_ai(model, few_shot, diseases)

In [38]:
def save_to_json(data, filename):
    """
    Salva um dicionário em um arquivo JSON.

    Args:
        data: O dicionário a ser salvo.
        filename: O nome do arquivo JSON.
    """
    try:
        with open(filename, "w") as json_file:
            json.dump(data, json_file, indent=4)
    except IOError as e:
        print(f"Erro ao salvar o arquivo '{filename}': {e}")

In [39]:
# Salva o dicionário em um arquivo JSON
save_to_json(diseases_symptoms_ai, "diseases_symptoms_medline_gemini.json")